# Deep dive into LangChain

In [1]:
pip install -r ./files/requirements.txt -q

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip show openai

Name: openai
Version: 1.97.1
Summary: The official Python library for the openai API
Home-page: https://github.com/openai/openai-python
Author: 
Author-email: OpenAI <support@openai.com>
License: Apache-2.0
Location: /opt/anaconda3/lib/python3.12/site-packages
Requires: anyio, distro, httpx, jiter, pydantic, sniffio, tqdm, typing-extensions
Required-by: langchain-openai
Note: you may need to restart the kernel to use updated packages.


## Python dotenv

In [3]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

os.environ.get('PORT')

'1337'

## Chat Models 

In [4]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()

output = llm.invoke('Explain LangChain in one sentence.')
print(output.content)

LangChain is a decentralized platform that uses blockchain technology to provide accurate and secure language translation services.


In [5]:
# help(ChatOpenAI)

In [6]:
from langchain.schema import (
    SystemMessage,
    AIMessage,
    HumanMessage
)
messages = [
    SystemMessage(content='You are a professional AI engineer with many years of experience'),
    HumanMessage(content='Explain the LangChain tool in a clear and simple way to a beginner in the field.')
]
output = llm.invoke(messages)
print(output.content)

Certainly! LangChain is a powerful tool used in natural language processing (NLP) tasks like text classification, sentiment analysis, and chatbots. It provides pre-trained models and tools to help developers build and deploy NLP applications quickly and easily.

With LangChain, beginners can easily create NLP models without needing to start from scratch. The tool offers a variety of functionalities such as tokenization, part-of-speech tagging, named entity recognition, and more. By using LangChain, developers can expedite the development process and focus more on customizing the models to fit their specific needs.

In essence, LangChain simplifies the development of NLP applications by providing ready-to-use tools and models, enabling beginners to create sophisticated language processing applications with ease.


## Caching LLM Response

### 1. In-Memory Cache

In [7]:
from langchain.globals import set_llm_cache
from langchain_openai import OpenAI
llm = OpenAI(model_name='gpt-3.5-turbo-instruct')

In [9]:
%%time
from langchain.cache import InMemoryCache

set_llm_cache(InMemoryCache())

# First request (not in cache, takes longer)
prompt = 'Tell me a joke a toddler can understand'
llm.invoke(prompt)

CPU times: user 15.3 ms, sys: 3.45 ms, total: 18.7 ms
Wall time: 1.59 s


'\n\nWhy did the tomato turn red?\n\nBecause it saw the salad dressing! '

In [10]:
%%time
# Second request (cached, faster)
llm.invoke(prompt)

CPU times: user 425 μs, sys: 1e+03 ns, total: 426 μs
Wall time: 440 μs


'\n\nWhy did the tomato turn red?\n\nBecause it saw the salad dressing! '

### 2. SQLite Caching

In [13]:
%%time
from langchain.cache import SQLiteCache

set_llm_cache(SQLiteCache(database_path='.langchain.db'))

# First request (not in cache, takes longer)
llm.invoke('Tell me a joke')

CPU times: user 5.57 ms, sys: 4.28 ms, total: 9.85 ms
Wall time: 7.35 ms


'\n\nWhy was the math book sad? Because it had too many problems.'

In [14]:
%%time
# Second request (cached, faster)
llm.invoke('Tell me a joke')

CPU times: user 2.95 ms, sys: 1.31 ms, total: 4.27 ms
Wall time: 3.21 ms


'\n\nWhy was the math book sad? Because it had too many problems.'

## LLM Streaming

In [15]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()
prompt = 'Write a rock song about the Moon and a Raven'
print(llm.invoke(prompt).content)

Verse 1:
Beneath the pale moonlight,
A raven takes flight,
Silhouetted against the stars,
Its caw echoing from afar.

Chorus:
Moon and raven, forever intertwined,
In the darkness, they both shine,
Guiding us through the night,
With their eerie, haunting light.

Verse 2:
The Moon, a silent observer,
Watching over us, forever,
While the raven soars high,
A messenger from the sky.

Chorus:
Moon and raven, forever intertwined,
In the darkness, they both shine,
Guiding us through the night,
With their eerie, haunting light.

Bridge:
With wings outstretched, the raven calls,
A symbol of mystery that enthralls,
While the Moon silently keeps watch,
A celestial guardian, never to botch.

Chorus:
Moon and raven, forever intertwined,
In the darkness, they both shine,
Guiding us through the night,
With their eerie, haunting light.

Outro:
So let the Moon and raven guide your way,
Through the darkness, they will stay,
A symbol of strength and grace,
In this chaotic, bewildering place.


In [16]:
# Enable streaming
for chunk in llm.stream(prompt):
    print(chunk.content, end='', flush=True)

Verse 1:
In the dead of night, in the pale moonlight
A raven swoops down, a shadow in flight
Its eyes are black, its feathers shine
A mysterious creature, of ominous design

Chorus:
Oh, Moon and Raven, dance in the sky
An eternal bond, never to die
Their spirits intertwined, forever entwined
In the darkness of the night, their secrets they'll find

Verse 2:
The Moon whispers secrets, to the Raven's ear
Of ancient tales of love and fear
They soar through the night, in a symphony of sound
Their connection unbreakable, forever bound

Chorus:
Oh, Moon and Raven, dance in the sky
An eternal bond, never to die
Their spirits intertwined, forever entwined
In the darkness of the night, their secrets they'll find

Bridge:
As the Raven cries out, to the silvery Moon
Their connection grows stronger, in the quiet of the gloom
They are creatures of the night, with stories untold
A bond that will never break, a tale to behold

Chorus:
Oh, Moon and Raven, dance in the sky
An eternal bond, never to die

## Templates

### PromptTemplate

In [17]:
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

template = '''You are an experienced hiring manager. 
Write a few sentences on what you look out for in a candidate's {tool1} and {tool2} on linkedin
'''

prompt_template = PromptTemplate.from_template(template=template)
prompt = prompt_template.format(tool1='resume', tool2='headline')
prompt

"You are an experienced hiring manager. \nWrite a few sentences on what you look out for in a candidate's resume and headline on linkedin\n"

In [18]:
llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0)
output = llm.invoke(prompt)
print(output.content)

When reviewing a candidate's resume, I look for a clear and concise summary of their experience, skills, and accomplishments. I also pay attention to the layout and formatting, as well as any relevant certifications or education. On LinkedIn, I look for a professional headline that accurately reflects the candidate's expertise and career goals. I also consider the quality of their profile picture and the content they share on their profile. Overall, I value candidates who present themselves in a professional and engaging manner.


### ChatPromptTemplate

In [19]:
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
#from langchain.prompts import SystemMessagePromptTemplate # For dynamic content in system message
from langchain_core.messages import SystemMessage # When dynamic content is not needed

chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content='You respond only in JSON format'),
        HumanMessagePromptTemplate.from_template('Top {n} countries in {area} by population')
    ]
)

messages = chat_template.format_messages(n='10', area='World')
print(messages)

[SystemMessage(content='You respond only in JSON format', additional_kwargs={}, response_metadata={}), HumanMessage(content='Top 10 countries in World by population', additional_kwargs={}, response_metadata={})]


In [20]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()
output = llm.invoke(messages)
print(output.content)

{
    "1": "China",
    "2": "India",
    "3": "United States",
    "4": "Indonesia",
    "5": "Pakistan",
    "6": "Brazil",
    "7": "Nigeria",
    "8": "Bangladesh",
    "9": "Russia",
    "10": "Mexico"
}


## Chains

### Simple Chains

In [21]:
from langchain_openai import ChatOpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain

llm = ChatOpenAI()
template = '''You are an experienced virologist.
Write a few sentences about the following virus "{virus}" in {language}.'''
prompt = PromptTemplate.from_template(template)

chain = prompt | llm

output = chain.invoke({'virus': 'HIV', 'language': 'English'})
print(output.content)

HIV, or human immunodeficiency virus, is a retrovirus that attacks the immune system, specifically targeting CD4 cells which are crucial for the body's ability to fight off infections. When left untreated, HIV can progress to acquired immunodeficiency syndrome (AIDS), a condition in which the immune system is severely weakened, leaving individuals susceptible to opportunistic infections and certain cancers. HIV is primarily transmitted through sexual contact, sharing needles, and from mother to child during childbirth or breastfeeding. Advances in antiretroviral therapy have greatly improved the prognosis for individuals living with HIV, turning what was once a fatal disease into a manageable chronic condition.


In [22]:
template = 'What is the capital of {country}?. List the top 3 places to visit in that city. Use bullet points'
prompt = PromptTemplate.from_template(template)

# the pattern used above is the latest pattern, this one has been deprecated but I want to see the 'Verbose' in action
chain = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True
)

# the latest pattern to be used
# from langchain_core.output_parsers import StrOutputParser
# chain = prompt_template | llm | StrOutputParser()

country = input('Enter Country: ')
output = chain.invoke(country)
print(output['text'])

/var/folders/_9/tsds32ps4619kckrn0jhpyjh0000gq/T/ipykernel_24544/2046776619.py:5: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(


Enter Country:  Nigeria




> Entering new LLMChain chain...
Prompt after formatting:
What is the capital of Nigeria?. List the top 3 places to visit in that city. Use bullet points

> Finished chain.
The capital of Nigeria is Abuja.

Top 3 places to visit in Abuja:
- Zuma Rock
- Aso Rock
- National Mosque


### Sequential Chains

In [23]:
from langchain_openai import ChatOpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain

llm1 = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0.5)
prompt_template1 = PromptTemplate.from_template(
    template='You are an experienced scientist and Python programmer. Write a function that implements the concept of {concept}.'
)
chain1 = LLMChain(llm=llm1, prompt=prompt_template1)

llm2 = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=1.2)
prompt_template2 = PromptTemplate.from_template(
    template='Given the Python function {function}, describe it as detailed as possible.'
)
chain2 = LLMChain(llm=llm2, prompt=prompt_template2)

overall_chain = SimpleSequentialChain(chains=[chain1, chain2], verbose=True)
output = overall_chain.invoke('Linear regression')



> Entering new SimpleSequentialChain chain...
Sure! Here is a simple implementation of linear regression in Python:

```python
import numpy as np

def linear_regression(X, y):
    # Add a column of ones to X for the intercept term
    X = np.hstack((np.ones((X.shape[0], 1)), X))
    
    # Calculate the coefficients using the normal equation
    theta = np.linalg.inv(X.T @ X) @ X.T @ y
    
    return theta

# Example usage
X = np.array([[1], [2], [3], [4]])
y = np.array([2, 4, 6, 8])

theta = linear_regression(X, y)
print(theta)
```

This function takes in the feature matrix `X` and the target vector `y`, adds a column of ones to `X` for the intercept term, and then calculates the coefficients `theta` using the normal equation. Finally, it returns the coefficients `theta` that define the linear regression model.
This Python code implements a simple linear regression model. Here's a detailed explanation of the code:

1. First, the necessary library `numpy` is imported to handle numer

In [24]:
print(output['output'])

This Python code implements a simple linear regression model. Here's a detailed explanation of the code:

1. First, the necessary library `numpy` is imported to handle numerical operations efficiently.

2. The `linear_regression` function is defined, which takes two parameters `X` and `y` representing the feature matrix (input data) and target vector (output data), respectively.

3. Inside the function, a column of ones is added to the feature matrix `X`. This is done to account for the intercept term in the linear regression model.

4. The coefficients `theta` are calculated using the normal equation:

    \[
    theta = (X^T X)^{-1} X^T y
    \]

    Without going into too much detail, this equation provides a closed-form solution to finding the optimal values of the coefficients that minimize the mean squared error of the linear regression model.

5. The calculated coefficients `theta` are returned from the function.

6. An example usage of the function is shown by creating a simple